In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
print('Pytorch version :', torch.__version__)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Device :',device)

Pytorch version : 1.7.1
Device : cuda:0


In [2]:
train_data = datasets.CIFAR100(root='data',train=True,transform=transforms.ToTensor(),download=True)
test_data = datasets.CIFAR100(root='data',train=False,transform=transforms.ToTensor(),download=True)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
train_data.__getitem__(0)[0].size()

torch.Size([3, 32, 32])

In [4]:
BATCH_SIZE = 512
train_iter = DataLoader(train_data,batch_size=BATCH_SIZE,shuffle=True,num_workers=1)
test_iter = DataLoader(test_data,batch_size=BATCH_SIZE,shuffle=True,num_workers=1)

In [5]:
import torch
import torch.nn as nn

class VGG(nn.Module):
    def __init__(self, num_classes=100):
        super(VGG, self).__init__()

        self.relu = nn.ReLU(inplace=True)

        # Convolution Feature Extraction Part
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1) # 64,width,height
        self.bn1   = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2) # 64, width/2, height/2

        '''==========================================================='''
        '''======================== TO DO (1) ========================'''
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1) # 128, width/2, height/2
        self.bn2   = nn.BatchNorm2d(128)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2) # 128, width/4, height/4

        self.conv3_1 = nn.Conv2d(128,256, kernel_size=3, padding=1) # 256, width/4, height/4
        self.bn3_1   = nn.BatchNorm2d(256)
        self.conv3_2 = nn.Conv2d(256,256, kernel_size=3, padding=1) # 256, width/4, height/4
        self.bn3_2   = nn.BatchNorm2d(256)
        self.pool3   = nn.MaxPool2d(kernel_size=2, stride=2) # 256, width/8, height/8

        self.conv4_1 = nn.Conv2d(256,512, kernel_size=3, padding=1) # 512, width/8, height/8
        self.bn4_1   = nn.BatchNorm2d(512)
        self.conv4_2 = nn.Conv2d(512,512, kernel_size=3, padding=1) # 512, width/8, height/8
        self.bn4_2   = nn.BatchNorm2d(512)
        self.pool4   = nn.MaxPool2d(kernel_size=2, stride=2) # 512, width/16, height/16

        self.conv5_1 = nn.Conv2d(512,512, kernel_size=3, padding=1) # 512, width/16, height/16
        self.bn5_1   = nn.BatchNorm2d(512)
        self.conv5_2 = nn.Conv2d(512,512, kernel_size=3, padding=1) # 512, width/16, height/16
        self.bn5_2   = nn.BatchNorm2d(512)
        self.pool5   = nn.MaxPool2d(kernel_size=2, stride=2) # 512, width/32, height/32
        '''======================== TO DO (1) ========================'''
        '''==========================================================='''


        # Fully Connected Classifier Part
        self.fc1      = nn.Linear(512 * 1 * 1, 4096)
        self.dropout1 = nn.Dropout()

        '''==========================================================='''
        '''======================== TO DO (2) ========================'''
        self.fc2      = nn.Linear(4096,4096)
        self.dropout2 = nn.Dropout()

        self.fc3      = nn.Linear(4096,100)
        '''======================== TO DO (2) ========================'''
        '''==========================================================='''

    def forward(self, x):
        # Convolution Feature Extraction Part
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.pool2(x)

        x = self.conv3_1(x)
        x = self.bn3_1(x)
        x = self.relu(x)
        x = self.conv3_2(x)
        x = self.bn3_2(x)
        x = self.relu(x)
        x = self.pool3(x)

        x = self.conv4_1(x)
        x = self.bn4_1(x)
        x = self.relu(x)
        x = self.conv4_2(x)
        x = self.bn4_2(x)
        x = self.relu(x)
        x = self.pool4(x)

        x = self.conv5_1(x)
        x = self.bn5_1(x)
        x = self.relu(x)
        x = self.conv5_2(x)
        x = self.bn5_2(x)
        x = self.relu(x)
        x = self.pool5(x)

        # Fully Connected Classifier Part
        x = torch.flatten(x,1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout1(x)

        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout2(x)

        x = self.fc3(x)
        return x

In [6]:
# Network
model = VGG(num_classes=100)

# Random input
x = torch.randn((1, 3, 32, 32))

# Forward
out = model(x)

# Check the output shape
print("Output tensor shape is :", out.shape)

Output tensor shape is : torch.Size([1, 100])


In [7]:
model = VGG().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=1e-4)

In [8]:
def eval_func(model,data_iter,device):
    model.eval()
    with torch.no_grad():
        count, total_count = 0,0
        for batch_in, batch_label in data_iter:
            x = batch_in.to(device)
            y = batch_label.to(device)
            y_pred = model.forward(x)
            y_ = torch.argmax(y_pred, dim=-1)
            count += (y==y_).sum().item()
            total_count += batch_in.size(0)
    model.train()
    return count/total_count

In [9]:
eval_func(model,train_iter,device)

0.01

In [23]:
num = torch.randint(0,8,(3,8,8)).type(torch.Tensor)
num

tensor([[[4., 2., 1., 2., 1., 3., 6., 0.],
         [1., 2., 0., 2., 1., 3., 5., 4.],
         [7., 4., 6., 7., 0., 1., 6., 6.],
         [4., 0., 7., 1., 3., 3., 5., 0.],
         [3., 0., 7., 1., 2., 5., 7., 6.],
         [3., 4., 2., 1., 0., 2., 7., 3.],
         [6., 4., 4., 0., 6., 6., 6., 7.],
         [4., 1., 5., 6., 4., 3., 6., 5.]],

        [[5., 2., 0., 3., 2., 4., 5., 3.],
         [0., 1., 2., 3., 2., 5., 0., 5.],
         [7., 4., 2., 1., 3., 6., 5., 0.],
         [0., 1., 2., 7., 6., 2., 2., 5.],
         [4., 6., 6., 4., 1., 2., 0., 5.],
         [5., 0., 4., 0., 2., 5., 1., 6.],
         [2., 4., 4., 7., 2., 1., 6., 3.],
         [1., 0., 3., 1., 5., 5., 2., 0.]],

        [[0., 3., 7., 2., 3., 0., 3., 4.],
         [6., 6., 0., 7., 0., 2., 2., 7.],
         [5., 0., 2., 7., 4., 5., 1., 4.],
         [1., 3., 3., 2., 7., 4., 2., 5.],
         [1., 2., 7., 0., 4., 3., 1., 2.],
         [4., 2., 2., 2., 2., 1., 5., 6.],
         [7., 3., 6., 3., 5., 3., 7., 3.],
       

In [26]:
a = nn.AdaptiveAvgPool2d((4,3))
a(num)

tensor([[[1.6667, 1.6250, 3.5000],
         [4.6667, 3.5000, 3.5000],
         [3.1667, 2.5000, 5.0000],
         [4.0000, 4.2500, 5.5000]],

        [[1.6667, 2.6250, 3.6667],
         [2.6667, 3.6250, 3.3333],
         [4.1667, 3.0000, 3.1667],
         [2.3333, 3.5000, 2.8333]],

        [[3.6667, 2.6250, 3.0000],
         [2.3333, 4.2500, 3.5000],
         [3.0000, 2.6250, 3.0000],
         [3.5000, 3.8750, 3.5000]]])

In [41]:
os.makedirs('./log', exist_ok=True)
EPOCH = 10
model.train()
for epoch in range(EPOCH):
    for batch_in, batch_label in train_iter:
        x = batch_in.to(device)
        y = batch_label.to(device)
        y_pred = model.forward(x)
        optimizer.zero_grad()
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()
    train_acc = eval_func(model,train_iter,device)
    test_acc = eval_func(model,test_iter,device)
    print(f'EPOCH : {epoch}, train_acc : {train_acc}, test_acc : {test_acc}')

EPOCH : 0, train_acc : 0.17022, test_acc : 0.1642
EPOCH : 1, train_acc : 0.30734, test_acc : 0.2763
EPOCH : 2, train_acc : 0.4009, test_acc : 0.3279
EPOCH : 3, train_acc : 0.50264, test_acc : 0.3613
EPOCH : 4, train_acc : 0.57984, test_acc : 0.364
EPOCH : 5, train_acc : 0.70262, test_acc : 0.3755
EPOCH : 6, train_acc : 0.73908, test_acc : 0.3554
EPOCH : 7, train_acc : 0.74216, test_acc : 0.3292
EPOCH : 8, train_acc : 0.85366, test_acc : 0.3596
EPOCH : 9, train_acc : 0.85792, test_acc : 0.3405


In [ ]:
num_choice = 25
choice = np.random.choice(60000,num_choice,replace=False)
x = train_data.data[choice]
t = train_data.target[choice]